# Práctica 2. Aprendizaje por refuerzo
Inteligencia Artificial I    2022/2023

Grupo 2

## Parte 1. Taxi
Vamos a utilizar como primer ejemplo un entorno de jugete de la librería Gym 

https://www.gymlibrary.dev/environments/toy_text/taxi/

Tenemos una rejilla rectangular con 4 ubicaciones (etiquetadas con letras diferentes) y nuestro trabajo es recoger al pasajero en una ubicación y dejarlo en otra. Se trata de llevar al pasajero que inicialmente está en Y (0,4) a la posición destino R (0,0). El taxi solo puede coger y dejar pasajeros en las posiciones marcadas.
Recibimos +20 puntos por dejar al pasajero con éxito y perderemos 1 punto por cada paso. También hay una penalización de 10 puntos por acciones ilegales de recoger y dejar.

En la librería OpenAI's Gym tenemos definido el modelo de este entorno de pruebas (Taxi) y podemos desarrollar y evaluar nuestro agente que aprende usando RL. La librería gym nos proporciona la representación y la visualización (render) del tablero por lo que no tenemos que hacerlo (aunque sería sencillo representarlo en AIMA y resolverlo con cualquiera de sus algoritmos de búsqueda). 

In [ ]:
# Para el ejemplo del taxi y el cart pool es necesario instalar algunas librerías (gym,cmake,scipy) si no están ya instaladas
# !pip install cmake
# La librería gym proporciona problemas de prueba — environments — con una interfaz general que puedes usar para probar 
# distintos algoritmos y configuraciones de RL. 

# !pip install gym
#!pip install ale-python-interface
#!pip install gym --force-reinstall --user
# tps://gym.openai.com/docs/

##!pip install scipy
import sys 
!{sys.executable} -m pip install cmake
import sys 
!{sys.executable} -m pip install scipy
import sys
!{sys.executable} -m pip install gym
import sys
!{sys.executable} -m pip install pygame --pre


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Jimmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Jimmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Jimmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Jimmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import gym
env = gym.make("Taxi-v3").env
# Taxi environment con la visualización por defecto
#env = gym.make('Taxi-v3',render_mode="human")
#env = gym.make('Taxi-v3',render_mode="rgb_array")
#env = gym.make('Taxi-v3',render_mode="ansi")
#env.render() 
# El render explícito da error en la ultima version de gym. 
# Hay que especificar el modo de render al hacer make y se hace automático. 

![image.png](attachment:image.png)

La interfaz principal del entorno gym es env. 
Vamos a utilizar los siguientes métodos de env:

- env.reset: restablece el entorno y devuelve un estado inicial aleatorio.

- env.step (acción): realiza un paso en el entorno. Devuelve:

      observación: estado siguiente del entorno al aplicar la acción
      recompensa: asociada a la acción en ese estado
      done: Indica fin del episodio por éxito (hemos recogido y dejado a un pasajero)
      info: información adicional como rendimiento y latencia para depuración
      
- env.render: renderiza el entorno (útil para visualizar el entorno)

In [ ]:
# creamos e inicializamos el entorno 
env = gym.make('Taxi-v3', render_mode="human")
observation = env.reset()
# ejecuto una acción aleatoria
action = env.action_space.sample()
env.step(action)
env.close() # IMPORTANTE

In [ ]:
# Al ejecutar este fragmento de código veremos 20 episodios del tablero del taxi. 
# En cada episodio se prueban 10 acciones aleatorias.
env = gym.make('Taxi-v3',render_mode="human")
for i_episode in range(20):
    observation = env.reset() # devuelve un estado inicial aleatorio.
    for t in range(10):
        #print("estado {}".format(observation))
        action = env.action_space.sample()  # acción aleatoria.
        #print("action {}".format(action))
        observation, reward, done, info, X = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break       
    print("Episodio terminado sin éxito")
env.close()

Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito
Episodio terminado sin éxito


In [ ]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


El Action Space tiene tamaño 6 y el State Space tiene tamaño 500. El algoritmo RL no necesita más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de forma única asignando un número único a cada estado posible, y RL aprende a elegir un número de acción de 0 a 5 donde:
   - 0 = sur
   - 1 = norte
   - 2 = este
   - 3 = oeste
   - 4 = recoger
   - 5 = dejar

Los 500 estados corresponden a una codificación de la ubicación del taxi, la ubicación del pasajero y la ubicación de destino.

El aprendizaje por refuerzo aprenderá un mapeo de estados con la acción óptima a realizar en ese estado. Para ello realizaremos un proceso de exploración, es decir, el agente explora el entorno y toma acciones basadas en las recompensas definidas en el entorno. La acción óptima para cada estado es la acción que tiene la **mayor recompensa acumulativa** a largo plazo según la fórmula vista para el Q-learning.

Dado el estado representado en la imagen anterior, vamos a codificar su estado y dárselo al entorno para que se renderice en Gym. 
Las filas y columnas se numeran de 0 a 4 y, como se ve en la imagen, tenemos el taxi en la fila 3, columna 1, nuestro pasajero está en la ubicación 2 y nuestro destino es la ubicación 0:  R (0), G (1), Y (2), B (3)

Usando el método de codificación de estado Taxi-v3, podemos hacer lo siguiente:

In [ ]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state

State: 328


In [ ]:
# La función print_frames la usaremos para visualizar los estados en modo texto.
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        #print(frame['frame'].getvalue())
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.2)

In [ ]:
env = gym.make('Taxi-v3',render_mode="ansi")
env.reset()
env.s = state
action = env.action_space.sample()
state, reward, done, info, x = env.step(action)
frame = [] 
frame.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        })


In [ ]:
# En este caso solo mostramos un estado
print_frames(frame)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

Timestep: 1
State: 212
Action: 4
Reward: -10


Podemos asertar las posiciones en el estado con `env.encode()` o usar los números entre 0 y 499 que son los 500 estados válidos en el espacio de estados. 

En la visualización anterior:
- El cuadrado representa el taxi, que es **amarillo sin pasajero y verde con pasajero**.
- La marca ("|") representa una pared que el taxi no puede cruzar.
- R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra **azul** representa la ubicación actual de **recogida** de pasajeros y la letra morada es el destino actual.

In [ ]:
env.s = 499
action = env.action_space.sample()
state, reward, done, info, x = env.step(action)
frame = [] 
frame.append({
        'frame': env.render(),
        'state': env.s,
        'action': action,
        'reward': reward
        })
print_frames(frame)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1
State: 499
Action: 5
Reward: -10


In [ ]:
# Hay una tabla (diccionario) P del entorno 
# Es equivalente a la matriz R que hemos visto en clase, pero tiene más informacion.
#podemos ver los valores de reward asignados por defecto al estado 328 que usamos como ejemplo
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

Este diccionario tiene la siguiente estructura {acción: [(probabilidad, próximo estado, recompensa, hecho)]}.

- El valor de 0-5 corresponde a las acciones (sur, norte, este, oeste, recogida, entrega) que el taxi puede realizar.
- La probabilidad es siempre 1.0 (en este entorno)
- `nextstate` muestra el estado siguiente usando la acción en este índice del diccionario.
- Las acciones de movimiento tienen por defecto una recompensa de -1 y pickup / dropoff tiene -10 **Si estuvimos en un estado donde el taxi tiene un pasajero y estamos en la posición destino correcta se vería una recompensa de 20** en la action dropoff (5))
- `done` se utiliza para indicarnos cuándo hemos dejado a un pasajero en el lugar correcto. Cada dejada de un pasajero (dropoff) con éxito es el final de un episodio.

Hay que tener en cuenta que si nuestro agente eligiera explorar la acción dos (2) en este estado, iría hacia el este y chocaría contra una pared. El código fuente ha hecho imposible mover el taxi a través de una pared, por lo que si el taxi elige esa acción, seguirá acumulando -1 penalizaciones, lo que afecta la recompensa a largo plazo.

![image.png](attachment:image.png)

### Paso 1. Resolvemos el problema sin aprendizaje por refuerzo

Una vez que hemos entendido el modelo del entorno, veamos qué pasaría si intentamos utilizar la fuerza bruta para resolver el problema sin RL.
Dado que tenemos nuestra tabla P para las recompensas predeterminadas en cada estado, podemos intentar que nuestro taxi navegue solo con eso.
Crearemos un bucle infinito que se ejecutará hasta que un pasajero llegue a un destino (un episodio), o en otras palabras, cuando la recompensa recibida sea 20.

El método `env.action_space.sample()` selecciona automáticamente una acción aleatoria del conjunto de todas las acciones posibles.


In [ ]:
action = env.action_space.sample()
action

0

In [ ]:
env = gym.make('Taxi-v3', render_mode="ansi")
#env.s = 328  # podemos fijar un estado inicial (por ejemplo el estado 328) o un estado aleatorio con reset
env.reset()

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info, x = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 226
Penalties incurred: 59


In [ ]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 226
State: 0
Action: 5
Reward: 20


El agente va a ciegas y utiliza miles de pasos de tiempo y realiza muchos drop-offs incorrectos para entregar un solo pasajero al destino correcto (cuando acierta de casualidad).  

> Os dejo como ejercicio opcional (en más abajo está) la resolución de este problema con cualquiera de los algoritmos de búsqueda de AIMA. Simplemente hay que definir la clase Problem haciendo llamadas a env. 
Usando la codificación numérica de los estados de este entorno y las 5 acciones que hemos visto haciendo una llamada a next_state, reward, done, info = env.step(action) 

Vamos a resolverlo aquí con Aprendizaje por refuerzo que empieza haciendo ciclos a ciegas y aprende de la experiencia pasada.  Cuando "por casualidad" acertamos el agente guarda en la memoria (en forma de recompensa) qué acción fue la mejor para cada estado. Así en el futuro elegirá esa acción. 

In [ ]:
import gym
env = gym.make("Taxi-v3").env
# Estado inicial aleatorio
env.reset() # reset environment to a new, random state   EStado inicial aleatorio 
# O estado inicial establecido 
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state

State: 328


In [ ]:
state = env.encode(3, 1, 2, 0)     # 3,1 son as coordenadas del taxi.  
# 2 y 0 son las posiciones del pasajero y dejada del pasajero que estan numeradas de 0..4
#  passenger index, destination index  (creo que para indicar que esta en el taxi se usa tambien .. mirar.)

action = env.action_space.sample()    # genera una acción aleatoria
# Recordamos las acciones: 0 = south; 1 = north; 2 = east; 3 = west; 4 = pickup; 5 = dropoff
next_state, reward, done, info, X = env.step(action) 
print("State:", state)
print("Action:", action)

State: 328
Action: 5


In [ ]:
#next_state = env.step(action)[0] 
print("Next State:", next_state)
env.s = next_state

Next State: 108


In [ ]:
action = 5   
# Recordamos las acciones: 0 = south; 1 = north; 2 = east; 3 = west; 4 = pickup; 5 = dropoff
next_state, reward, done, info, X = env.step(action) 
print("State:", state)
print("Next Action:", action)

State: 328
Next Action: 5


In [ ]:
print("Next State:", next_state)
env.s = next_state

# como hacemos dropoff de un taxi vacio no hace nada

Next State: 108


In [ ]:
env.s = 300
state = env.s
next_state = 0
action = 3   
next_state, reward, done, info,X = env.step(action) 
print("State:", state)
print("Next Action:", action)
print("Next State:", next_state)

State: 300
Next Action: 3
Next State: 108


### Aprendizaje por refuerzo

Vamos a utilizar el algoritmo de Q-learning que hemos visto en clase y que le dará a nuestro agente algo de memoria.
Básicamente, Q-learning permite al agente utilizar las recompensas del entorno para aprender, con el tiempo, la mejor acción a realizar en un estado determinado.
En nuestro entorno de Taxi, tenemos **la tabla de recompensas**, P, de la que el agente aprenderá. Lo hace buscando recibir una recompensa por realizar una acción en el estado actual y luego actualizar un valor Q para recordar si esa acción fue beneficiosa.
Los valores almacenados en la tabla Q se denominan valores Q y se asignan a una combinación (estado, acción).
Un valor Q para una combinación de acción de estado particular es representativo de la "calidad" de una acción tomada desde ese estado. Mejores valores Q implican mejores posibilidades de obtener mayores recompensas.
Por ejemplo, si el taxi se enfrenta a un estado que incluye a un pasajero en su ubicación actual, es muy probable que el valor Q para la recogida sea más alto en comparación con otras acciones, como la bajada o el norte.
Los valores Q se inicializan a un valor arbitrario y, a medida que el agente se expone al entorno y recibe diferentes recompensas al ejecutar diferentes acciones, los valores Q se actualizan mediante la ecuación:

Q (estado, acción) ← (1 − α) Q (estado, acción) + α (recompensa + γ max Q (siguiente estado, todas las acciones))

Dónde:
- α (alfa) es la tasa de aprendizaje (0 <α≤1) 
     - Al igual que en los entornos de aprendizaje supervisado, α es la medida en que nuestros valores Q se actualizan en cada iteración.
- γ (gamma) es el factor de descuento (0≤γ≤1) 
    - Determina cuánta importancia queremos dar a las recompensas futuras. Un valor alto para el factor de descuento (cercano a 1) captura la recompensa efectiva a largo plazo, mientras que un factor de descuento de 0 hace que nuestro agente considere solo la recompensa inmediata.

En la fórmula anterior estamos asignando (←), o actualizando, el valor Q del estado actual y la acción del agente tomando primero un peso (1 − α) del antiguo valor Q y luego agregando el valor aprendido. El valor aprendido es una combinación de la recompensa por realizar la acción actual en el estado actual y la recompensa máxima descontada del siguiente estado en el que estaremos una vez que realicemos la acción actual.
Básicamente, estamos aprendiendo la acción adecuada a tomar en el estado actual al observar la recompensa por la combinación estado / acción actual y las recompensas máximas para el siguiente estado. Esto eventualmente hará que nuestro taxi considere la ruta con las mejores recompensas.
El valor Q de un par estado-acción es la suma de la recompensa instantánea y la recompensa futura descontada (del estado resultante). La forma en que almacenamos los valores Q para cada estado y acción es a través de una tabla Q

La tabla Q es una matriz donde tenemos una fila para cada estado (500) y una columna para cada acción (6). Primero se inicializa a 0 y luego los valores se actualizan después del entrenamiento. Tenga en cuenta que la Q-table tiene las mismas dimensiones que la mesa de recompensas, pero tiene un propósito completamente diferente.

En la siguiente figura los valores de la Q-Table se inicializan a cero y se van actualizando durante el aprendizaje.  Los valores optimizan el recorrido del agente a traves del entorno buscando las máximas recompensas. 

![image.png](attachment:image.png)

### Resumen del proceso de Q-Learning

    • Inicializar la tabla Q a todo ceros.
    • Comenzar a explorar acciones: para cada estado, seleccione cualquiera de las posibles acciones para el estado actual (S).
    • Ir al siguiente estado (S') como resultado de esa acción (a).
    • Para todas las acciones posibles del estado (S'), seleccione la que tenga el valor Q más alto.
    • Actualizar los valores de la tabla Q utilizando la ecuación.
    • Establecer el siguiente estado como el estado actual.
    • Si se alcanza el estado objetivo, finalizar y repetir el proceso.

### Explotación de valores aprendidos

Después de una fase de exploración aleatoria de acciones, los valores Q tienden a converger sirviendo a nuestro agente como una función de valor de acción que puede explotar para elegir la acción mejor para un estado dado.
Existe una compensación entre exploración (elegir una acción aleatoria) y explotación (elegir acciones basadas en valores Q ya aprendidos). Queremos evitar que la acción siga siempre la misma ruta y posiblemente se sobreajuste, por lo que introduciremos otro parámetro llamado ϵ "épsilon" para atender esto durante el entrenamiento.
En lugar de simplemente seleccionar la acción de valor Q mejor aprendida, a veces preferimos explorar más el espacio de acción. Un valor de épsilon más bajo da como resultado episodios con más penalizaciones (en promedio), lo cual es obvio porque estamos explorando y tomando decisiones al azar.

In [ ]:
# Primero se inicializa la Q-table a 500×6 matrix of zeros:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

Ejecutar el aprendizaje es un proceso lento pero se realiza una única vez. 
Una vez entrenado, podemos resolver cualquier problema de este entorno simplemente consultando la tabla y eligiendo la acción que maximiza la recompensa en cada paso (explotación). 
El algoritmo de entrenamiento que actualizará esta Q-table a medida que el agente explora el entorno durante miles de episodios. 

En la primera parte del bucle (`while not done`) decidimos si elegir una acción aleatoria o explotar los valores Q ya calculados. Esto se hace simplemente usando el valor épsilon y comparándolo con la función `random.uniform(0, 1)`, que devuelve un número arbitrario entre 0 y 1.
Ejecutamos la acción elegida en el entorno para obtener el `next_state` y la recompensa por realizar la acción. Después de eso, calculamos el valor Q máximo para las acciones correspondientes al `next_state`, y con eso, podemos actualizar fácilmente nuestro valor Q al `new_q_value`:

In [ ]:
env.reset()
state = env.s
state

300

In [ ]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    env.reset()
    state = env.s

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info, X = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10: # Pickup/dropoff illegally
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        #clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")


Episode: 100
Episode: 200
Episode: 300
Episode: 400
Episode: 500
Episode: 600
Episode: 700
Episode: 800
Episode: 900
Episode: 1000
Episode: 1100
Episode: 1200
Episode: 1300
Episode: 1400
Episode: 1500
Episode: 1600
Episode: 1700
Episode: 1800
Episode: 1900
Episode: 2000
Episode: 2100
Episode: 2200
Episode: 2300
Episode: 2400
Episode: 2500
Episode: 2600
Episode: 2700
Episode: 2800
Episode: 2900
Episode: 3000
Episode: 3100
Episode: 3200
Episode: 3300
Episode: 3400
Episode: 3500
Episode: 3600
Episode: 3700
Episode: 3800
Episode: 3900
Episode: 4000
Episode: 4100
Episode: 4200
Episode: 4300
Episode: 4400
Episode: 4500
Episode: 4600
Episode: 4700
Episode: 4800
Episode: 4900
Episode: 5000
Episode: 5100
Episode: 5200
Episode: 5300
Episode: 5400
Episode: 5500
Episode: 5600
Episode: 5700
Episode: 5800
Episode: 5900
Episode: 6000
Episode: 6100
Episode: 6200
Episode: 6300
Episode: 6400
Episode: 6500
Episode: 6600
Episode: 6700
Episode: 6800
Episode: 6900
Episode: 7000
Episode: 7100
Episode: 7200
E

In [ ]:
# La tabla Q ha cambiado despues de 100000 episodios. Vamos a ver cuales son los Q-values aprendidos en el estado de ejemplo. 
q_table[328]

array([ -2.404801  ,  -2.27325184,  -2.39086324,  -2.35796909,
       -10.5270043 , -11.16750237])

![image.png](attachment:image.png)

> Observa el valor máximo de Q en el estado 328. ¿Te parece que Q-learning ha aprendido efectivamente la mejor acción a realizar en este estado?

En este caso, por la representación visual del estado 328, podemos deducir que la acción de subirse arriba es lo mejor a realizar, para acercarse al punto Y que es donde se encuentra el pasajero. En comparación, la acción de irse a la izquierda es la peor, ya que se choca con la pared y no hace nada. El movimiento de irse a la derecha y abajo son permitidos, sin embargo, no acerca al taxi al pasajero, por lo que no son las mejores acciones a realizar.

Por último, hacer pickup / dropoff no tienen sentido en este estado, pues el taxi no está en el punto de recogida ni el destino con el pasajero.

Basado en el análisis anterior, veamos la tabla Q para el estado 328:

```
array([ -2.404801  ,  -2.27325184,  -2.39086324,  -2.35796909,
       -10.5270043 , -11.16750237])
```

Traduciéndolo a acciones, tenemos:

   - 0 = sur - -2.404801
   - 1 = norte - -2.27325184
   - 2 = este - -2.39086324
   - 3 = oeste - -2.35796909
   - 4 = recoger - -10.5270043
   - 5 = dejar - -11.16750237

Se puede ver que la **acción de subir (ir al norte) es la mejor** (la que tiene mayor valor), seguida de ir a la izquierda (oeste). Las acciones de recoger y dejar son los peores.

Por lo tanto, se puede decir que el algoritmo Q-learning ha aprendido correctamente la mejor acción a realizar en este estado.

### Evaluar el comportamiento del agente despues del proceso de Q-learning 

Para evaluar el comportamiento de nuestro agente no necesitamos explorar más acciones. En el comportamiento del agente ahora la siguiente acción siempre se selecciona utilizando el mejor valor Q:

In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 100
env = gym.make("Taxi-v3").env

for _ in range(episodes):
    env.reset()
    state = env.s
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info, X = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.76
Average penalties per episode: 0.0


Podemos ver en la evaluación que el comportamiento del agente ha mejorado significativamente y no hay penalizaciones, lo que significa que realizó las acciones correctas de recogida / devolución con 100 pasajeros diferentes.

Comparando los dos agentes vemos como aunque con Q-learning el agente comete errores inicialmente durante la exploración, una vez que ha explorado lo suficiente (visto la mayoría de los estados), puede actuar sabiamente maximizando las recompensas haciendo movimientos inteligentes. 

Veamos cuánto mejor es nuestra solución de Q-learning en comparación con el agente que realiza movimientos aleatorios.

Evaluamos a nuestros agentes de acuerdo con las siguientes métricas,

- **Número medio de penalizaciones por episodio**: Cuanto menor sea el número, mejor será el desempeño de nuestro agente. Idealmente, nos gustaría que esta métrica fuera cero o muy cercana a cero.

- **Número promedio de pasos de tiempo por viaje**: también queremos un número pequeño de pasos de tiempo por episodio, ya que queremos que nuestro agente dé pasos mínimos (es decir, el camino más corto) para llegar al destino.

- **Promedio de recompensas por movimiento**: cuanto mayor sea la recompensa, significa que el agente está haciendo lo correcto. Es por eso que **decidir las recompensas es una parte crucial del aprendizaje por refuerzo**. En nuestro caso, dado que tanto los tiempos como las penalizaciones se recompensan negativamente, una recompensa promedio más alta significaría que el agente llega al destino lo más rápido posible con la menor cantidad de penalizaciones.


    Average rewards per move	-3.9012092102214075	0.6962843295638126  
    Average number of penalties per episode	920.45	0.0  
    Average number of timesteps per trip	2848.14	12.38  

Estas métricas se calcularon en más de 100 episodios. 
Y como muestran los resultados, nuestro agente de Q-learning tiene un buen comportamiento. 


### Hiperparámetros y optimizaciones

Los valores de `alpha`,` gamma` y `epsilon` que hemos utilizado han sido elegidos por intuición, prueba y error pero hay mejores formas de obtener buenos valores. Idealmente, los tres deberían disminuir con el tiempo porque a medida que el agente continúa aprendiendo, en realidad construye antecedentes más válidos y duraderos;

    • α: (la tasa de aprendizaje) debería disminuir a medida que continúa adquiriendo una base de conocimientos cada vez mayor.
    • γ: a medida que se acerca cada vez más al valor límite, su preferencia por la recompensa a corto plazo debería aumentar, ya que no estará el tiempo suficiente para obtener la recompensa a largo plazo, lo que significa que su gamma debería disminuir.
    • ϵ: a medida que desarrollamos nuestra estrategia, tenemos menos necesidad de exploración y más explotación para obtener más utilidad de nuestra política, por lo que en vez de utilizar un valor fijo, a medida que aumentan los ensayos, épsilon debería disminuir. Al principio un épsilon alto genera más episodios de exploración y al final un épsilon bajo explota el conocimiento aprendido.

Una forma de obtener la combinación correcta de valores de hiperparámetros sería usar optimización local con algoritmos genéticos (que veremos en el próximo tema). 

Q-learning es uno de los algoritmos de aprendizaje por refuerzo más fáciles. Sin embargo, el problema con la obtención de Q es que, una vez que el número de estados en el entorno es muy alto, se vuelve difícil implementarlos con la tabla Q, ya que el tamaño sería muy, muy grande. Por eso se utilizan redes neuronales profundas en lugar de Q-table (Deep Reinforcement Learning). La red neuronal recibe información de estado y acciones en la capa de entrada y aprende a generar la acción correcta a lo largo del tiempo. Las técnicas de aprendizaje profundo (como las redes neuronales convolucionales) también se utilizan para interpretar los píxeles en la pantalla y extraer información del juego (como puntuaciones), y luego dejar que el agente controle el juego.

### Ejercicios   

#### Ejercicio 1

En el problema del Taxi resuelto con Q-Learning se pide experimentar de forma sistemática con distintos estados iniciales, distintos valores de los hiperparámetros y con distinto números de episodios de aprendizaje y comentar los resultados obtenidos. 
Observa explícitamente el comportamiento del agente con los valores límite de los hiperparámetros. 
Comenta de forma razonada las conclusiones obtenidas de los distintos procesos de aprendizaje. 
Evalua y compara los agentes respecto a las métricas dadas.

Para ello definimos una función auxiliar para automatizar el proceso.

In [ ]:
import numpy as np
def evaluar_problema_taxi(q_table):
    total_epochs, total_penalties = 0, 0
    episodes = 200
    env = gym.make("Taxi-v3").env

    # Probar el rendimiento del Q table con 100 episodios aleatorios
    for _ in range(episodes):
        env.reset()
        state = env.s
        epochs, penalties, reward = 0, 0, 0

        done = False

        while not done:
            action = np.argmax(q_table[state])
            state, reward, done, info, X = env.step(action)

            if reward == -10:
                penalties += 1

            epochs += 1

        total_penalties += penalties
        total_epochs += epochs

    print(f"  Results after {episodes} episodes:")
    print(f"  Average timesteps per episode: {total_epochs / episodes}")
    print(f"  Average penalties per episode: {total_penalties / episodes}")

# Probamos entrenar el problema de Taxi con distintos valores de alpha, gamma y epsilon. Y sacamos su metrica
def probar_problema_taxi(alpha, gamma, epsilon, rounds = 10_000):
    import random
    from IPython.display import clear_output
    
    # Crear problema, con un estado inicial aleatorio
    env = gym.make("Taxi-v3", render_mode="ansi")
    env.reset()
    
    # Primero se inicializa la Q-table a 500×6 matrix of zeros:
    q_table = np.zeros([env.observation_space.n, env.action_space.n])

    # For plotting metrics
    all_epochs = []
    all_penalties = []
    
    initial_state = env.s

    for i in range(1, rounds):
        env.reset()
        state = env.s

        epochs, penalties, reward, = 0, 0, 0
        done = False

        while not done:
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values

            next_state, reward, done, info, X = env.step(action) 

            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            if reward == -10: # Pickup/dropoff illegally
                penalties += 1

            state = next_state
            epochs += 1

    print(f"Training finished for initial state {initial_state} with alpha = {alpha}, gamma = {gamma}, epsilon = {epsilon}.")
    
    # Imprimir el contenido de una celda aleatoria de la tabla Q
    print(f"  Q-table for initial state {initial_state} with alpha = {alpha}, gamma = {gamma}, epsilon = {epsilon}:")
    print(f"  {q_table[initial_state]}")

    evaluar_problema_taxi(q_table)
    print()

Para analizar el comportamiento al variar los hiperparámetros, nos centramos en valores extremos, pues sería donde se ve más el cambio del comportamiento.

Recordando el significado de los hiperparámetros:

- **alpha**: es la tasa de aprendizaje, es decir, la importancia que se le da a la nueva información. Si es muy pequeña, el agente aprende poco.
- **gamma**: es la importancia que se le da a la recompensa futura. Si es muy pequeña, el agente no se fija en la recompensa futura, si es muy grande, el agente se fija demasiado en la recompensa futura y no se fija en el entorno.
- **epsilon**: es la probabilidad de explorar. Si es muy pequeña, el agente no explora, si es muy grande, el agente explora demasiado y no se fija en el entorno.

Así, vamos a explorar siguientes posibilidads:

1. Valor límite de alpha: 0.1 y 0.9
   Con estos valores, el agente aprende muy poco o muy rápido, respectivamente.

2. Valor límite de gamma: 0.1 y 0.9
   Con estos valores, el agente no se fija en la recompensa futura o se fija demasiado, respectivamente.

3. Valor límite de epsilon: 0.1 y 0.9
   Con estos valores, el agente no explora o explora demasiado, respectivamente.

Cuando probamos con el valor extremo de un hiperparámetro, los otros dos hiperparámetros se ponen a valores medios (0.5).

### 1. Valores extremos de alpha

In [ ]:
probar_problema_taxi(alpha=0.1, gamma=0.5, epsilon=0.5)

In [ ]:
probar_problema_taxi(alpha=0.9, gamma=0.5, epsilon=0.5)

Training finished for initial state 234 with alpha = 0.9, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 234 with alpha = 0.9, gamma = 0.5, epsilon = 0.5:
  [ -1.99865723  -1.99865723  -1.99462891  -1.99865723 -10.99731445
 -10.99731445]
  Results after 200 episodes:
  Average timesteps per episode: 13.065
  Average penalties per episode: 0.0



El caso de prueba con `alpha = 0.9` se ha podido terminar con éxito, mientras que el caso de prueba con `alpha = 0.1` no ha podido terminar después de 70 minutos. 

Al tomar alpha muy pequeño, el comportamiento del agente es muy parecido al de un agente aleatorio, y por eso tardaría más en aprender también. Pero la lentitud del aprendizaje no es el causante de que no se haya podido terminar la prueba en este caso.

La conclusión que hemos sacado es que, a pesar de que se ha terminado el entrenamiento de Q Table (se puede ver en el mensaje de salida), como el valor de `alpha` es muy pequeño, el agente aprende muy poco, entonces el **resultado del Q table es poco fiable**. Por lo que al simular con episodios aleatorios, las acciones tomadas en cada paso, como sólo depende del valor de Q table, puede meterse en un bucle infinito si está realizando siempre un movimiento inválido que no le lleva a ningún sitio.


### 2. Valores extremos de gamma

In [ ]:
probar_problema_taxi(alpha=0.5, gamma=0.1, epsilon=0.5)

Training finished for initial state 7 with alpha = 0.5, gamma = 0.1, epsilon = 0.5.
  Q-table for initial state 7 with alpha = 0.5, gamma = 0.1, epsilon = 0.5:
  [ -1.11111111  -1.11111111  -1.11111111  -1.11111111 -10.11111048
 -10.11103301]
  Results after 200 episodes:
  Average timesteps per episode: 13.07
  Average penalties per episode: 0.0



In [ ]:
probar_problema_taxi(alpha=0.5, gamma=0.9, epsilon=0.5)

Training finished for initial state 489 with alpha = 0.5, gamma = 0.9, epsilon = 0.5.
  Q-table for initial state 489 with alpha = 0.5, gamma = 0.9, epsilon = 0.5:
  [ -5.4982769   -4.99684549  -5.53210613  -5.02621403 -14.62641713
 -14.76271598]
  Results after 200 episodes:
  Average timesteps per episode: 13.01
  Average penalties per episode: 0.0



Al tomar valores extremos de gamma, el agente se fija más o menos en la recompensa futura. 

En este caso, podemos observar que el contenido de Q table para `gamma = 0.1` tiene valores repetidos (hemos realizado varias ejecuciones para ver que no es un caso particular). Por ejemplo, en una de las ejecuciones ha salido:

```
Training finished for initial state 261 with alpha = 0.5, gamma = 0.1, epsilon = 0.5.
  Q-table for initial state 261 with alpha = 0.5, gamma = 0.1, epsilon = 0.5:
  [ -1.11111111  -1.11111111  -1.11111111  -1.11111111 -10.11111111
 -10.11111111]
```

Si observamos la fórmula para actualizar Q table: `Q(s, a) = (1 - alpha) * Q(s, a) + alpha * (r + gamma * max(Q(s', a')))`, vemos que si el valor de `gamma` es muy pequeño, tenemos muy poco en cuenta la recompensa futura, y solo tenemos en cuenta la recompensa actual (`r`). 

Como la recompensa actual de movimiento es por defecto **-1**, y de pickup y dropoff es **-10**, al operar con el parámetro `alpha`, salen esos valores tan parecidos.

En cuanto a los tiempos por episodio (lo que tarda en entregar el viajero), tras ensayar con 200 episodios aleatorios, se puede ver que no son tan diferentes. En una de las ejecuciones, se ha obtenido:

```
Training finished for initial state 7 with alpha = 0.5, gamma = 0.1, epsilon = 0.5.
  Q-table for initial state 7 with alpha = 0.5, gamma = 0.1, epsilon = 0.5:
  [ -1.11111111  -1.11111111  -1.11111111  -1.11111111 -10.11111048
 -10.11103301]
  Results after 200 episodes:
  Average timesteps per episode: 13.07
  Average penalties per episode: 0.0

Training finished for initial state 489 with alpha = 0.5, gamma = 0.9, epsilon = 0.5.
  Q-table for initial state 489 with alpha = 0.5, gamma = 0.9, epsilon = 0.5:
  [ -5.4982769   -4.99684549  -5.53210613  -5.02621403 -14.62641713
 -14.76271598]
  Results after 200 episodes:
  Average timesteps per episode: 13.01
  Average penalties per episode: 0.0
```

### 3. Valores extremos de epsilon

In [ ]:
probar_problema_taxi(alpha=0.5, gamma=0.5, epsilon=0.1)

Training finished for initial state 91 with alpha = 0.5, gamma = 0.5, epsilon = 0.1.
  Q-table for initial state 91 with alpha = 0.5, gamma = 0.5, epsilon = 0.1:
  [ -1.99966405  -1.99973387  -1.99978189  -1.99966416 -10.19455745
  -7.99954751]
  Results after 200 episodes:
  Average timesteps per episode: 13.02
  Average penalties per episode: 0.0



In [ ]:
probar_problema_taxi(alpha=0.5, gamma=0.5, epsilon=0.9)

Training finished for initial state 114 with alpha = 0.5, gamma = 0.5, epsilon = 0.9.
  Q-table for initial state 114 with alpha = 0.5, gamma = 0.5, epsilon = 0.9:
  [ -1.99865723  -1.99966431  -1.99865723  -1.99932861 -10.99932861
 -10.99932861]
  Results after 200 episodes:
  Average timesteps per episode: 13.29
  Average penalties per episode: 0.0



Al tomar valores extremos de epsilon, el agente explora más o menos. Esto se puede ver reflejado claramente en el **tiempo de ejecución**. 

Para valor de epsilon = 0.1 (explora muy poco y se apoya en resultados de Q table), el tiempo de ejecución total ha sido de sólo 5.7 segundos. Mientras que para valor de epsilon = 0.9 (explora mucho más), el tiempo de ejecución ha sido de 1 minuto y 13 segundos, notablemente más lento en comparación con el caso anterior.

En cuanto al tiempo por episodio, tras ensayar con 200 episodios aleatorios, se puede ver que no son tan diferentes.

-------------------------

#### Ejercicio 2. Opcional
Resuelve el problema del taxi con cualquiera de los algoritmos de búsqueda de AIMA. Para ello tendrás que definir una subclase de Problem en la que los estados utilicen la codificación numérica de los 500 estados del entorno env de Gym con las 6 acciones.
    - 0: move south
    - 1: move north
    - 2: move east
    - 3: move west
    - 4: pickup passenger
    - 5: drop off passenger

En el código de la clase Problem tendrás que hacer uso de las funciones de env, por ejemplo, result corresponde a una llamada a:  env.next_state, reward, done, info, X = env.step(action)

##### Solución propuesta

Vamos a crear una clase Estado como hicimos con el problema de los muelles que nos va a permitir utilizar métodos que recurran a hash, y así poder llevar cuenta de los nodos por los que hemos pasado y no entrar en ciclos dentro del árbol de búsqueda. En el estado almacenamos si llevamos al pasajero en el taxi y si el pasajero se encuentra en el hotel, también guardamos la posición del taxi y la del pasajero, pues se ven modificadas a lo largo de la búsqueda.

In [ ]:
!export PYTHONPATH="$PYTHONPATH:../workspace_files/"


class Estado:
    def __init__(self, taxi, passenger):
        self.passengerInTaxi = False
        self.passengerInHotel = False
        self.taxiPosition = taxi
        self.passengerPosition = passenger
    
    def __lt__(self,other):
         return self.taxiPosition < other.taxiPosition
        
    def __str__(self):
        return f"Taxi: {self.taxiPosition} - Pasajero: {self.passengerPosition} \n"

In [ ]:
class HashableList(list):
    def __lt__(self,other):
         return tuple(self) < tuple(other)
    def __hash__(self):
        return hash(tuple(self))

Ahora definimos la subclase de Problem, que tiene como atributos el número de filas y columnas y la posición del hotel, pues no cambian a lo largo del soluciones. Definimos las acciones posibles cuidando las restricciones propias del problema, como no salirnos del tablero o solo coger al pasajero cuando corresponde. 

Las hemos codificado con su nombre en vez de con números porque nos parecía más intuitivo a la hora de visualizar la solución. La meta se alcanza cuando dejamos al pasajero en el hotel. Como heurística hemos considerado la distancia manhattan desde el taxi al pasajero sumada con la distancia manhattan del taxi al hotel.

In [ ]:
from copy import deepcopy
from search import *

import math

class ProblemaTaxi(Problem):
    def __init__(self, initial: Estado, hotelPos):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        super().__init__(initial)
        self.numFilas = 5
        self.numColumnas = 5
        self.hotelPosition = hotelPos
    
    def actions(self, state: Estado) -> (int, int):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        actions = ["pickup", "dropoff"]

        #Los movimiento que puede llevar a cabo el taxi son, norte, sur, oeste, este, pickup y dropoff
        if(state.taxiPosition[0] < self.numColumnas):
            actions.append("este")
        if(state.taxiPosition[0] > 0):
            actions.append("oeste")
        if(state.taxiPosition[1] < self.numFilas):
            actions.append("norte")
        if(state.taxiPosition[1] > 0):
            actions.append("sur")
            
        return actions

    def result(self, state: Estado, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        mov = [0,0]
        el = []
        res = deepcopy(state)

        if action == "norte":
            mov = [0, 1]
        elif action == "sur":
            mov = [0, -1]
        elif action == "oeste":
            mov = [-1, 0]
        elif action == "este":
            mov = [1, 0]
        elif action == "pickup" and state.taxiPosition == state.passengerPosition and (not state.passengerInTaxi):
            res.passengerInTaxi = True
        elif action == "dropoff" and state.passengerInTaxi :
            if state.taxiPosition == self.hotelPosition : 
                res.passengerInHotel = True

            res.passengerInTaxi = False
            
            
        for x, y in zip(res.taxiPosition, mov):
            el = el + [x + y]
                
        res.taxiPosition = el
        
        return res

    def goal_test(self, state: Estado):
        """Return True if the state is a goal."""
        return state.passengerInHotel
    
    def heuristica(self, node):
        state = node.state
        coste = []
        for x, y in zip(state.taxiPosition, self.hotelPosition):
            coste = coste + [abs(x + y)]
        return sum(coste)

Consideremos ahora un estado inicial válido.

In [ ]:
initial = Estado(HashableList([3,4]), HashableList([2,3]))
print(initial)

Taxi: [3, 4] - Pasajero: [2, 3] 



Creemos un problema asociado, asignando la posición del hotel de manera arbitraria.


In [ ]:
prob = ProblemaTaxi(initial, HashableList([1,2]))

Comprobamos que funciona correctamente:

Las dos casilla de abajo las tengo que borrar


In [ ]:
acts = prob.actions(initial)
print(acts[1])

dropoff


In [ ]:
print(prob.result(initial, acts[1]))

Taxi: [3, 4] - Pasajero: [2, 3] 



Creamos una clase que nos va a servir para llevar la cuenta de los repetidos, también creamos un método para que llame directamente a la búsqueda y nos muestre el resultado directamente.



In [ ]:
class Problema_con_Analizados(Problem):
    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda."""

    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado, accion)

In [ ]:
def resuelve_taxi(p, algoritmo, h=None):
    pa = Problema_con_Analizados(p)
    if h:
        sol = algoritmo(pa, h).solution()
    else:
        sol = algoritmo(pa).solution()
    print(f"Solución: {sol}")
    print(f"Algoritmo: {algoritmo.__name__}")
    if h:
        print(f"Heurística: {h.__name__}")
    print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol), pa.analizados))


Probemos con el método de búsqueda A*, vemos que obtenemos una solución bastante eficiente que cumple con las condiciones impuestas. Hemos probado con otras formas de búsqueda, como en anchura y en profundidad, pero hemos obtenido mejores resultado con ésta que presentamos.

In [ ]:
resuelve_taxi(prob, astar_search, prob.heuristica)

Solución: ['sur', 'oeste', 'pickup', 'oeste', 'sur', 'dropoff']
Algoritmo: astar_search
Heurística: heuristica
Longitud de la solución: 6. Nodos analizados: 7700


Como observamos hemos obtenido una solución de longitud 6 y hemos tenido que analizar 7700 nodos.

-----------------

# Parte 2. Uso de entrenamiento con QLearning para otro entorno de Gym

Realizar el mismo proceso de entrenamiento con QLearning para otro entorno de OpenAI Gym

## CartPole

Aunque se incluye para comentarlo en clase, este dominio no lo vamos a resolver porque utiliza valores continuos y no es posible entrenarlo con QLearning que usa valores discretos. 

https://gym.openai.com/envs/CartPole-v1/  

Un poste está unido por una articulación no accionada a un carro, que se mueve a lo largo de una pista sin fricción. El sistema se controla aplicando una fuerza de +1 o -1 al carro. El péndulo comienza en posición vertical y el objetivo es evitar que se caiga. Se proporciona una recompensa de +1 por cada paso de tiempo que el poste permanece en posición vertical. El episodio termina cuando el poste está a más de 15 grados de la vertical o el carro se mueve más de 2.4 unidades desde el centro.

In [ ]:
# Generamos el entorno CartPole v1 y realizamos acciones aleatorias. 
# Se trata de maximizar el tiempo que el palo está en equilibrio.

env = gym.make("CartPole-v1", render_mode="human")
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info, X = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break            
env.close()

Episode finished after 13 timesteps
Episode finished after 30 timesteps
Episode finished after 46 timesteps
Episode finished after 11 timesteps
Episode finished after 45 timesteps
Episode finished after 30 timesteps
Episode finished after 26 timesteps
Episode finished after 17 timesteps
Episode finished after 16 timesteps
Episode finished after 94 timesteps
Episode finished after 22 timesteps
Episode finished after 16 timesteps
Episode finished after 22 timesteps
Episode finished after 15 timesteps
Episode finished after 10 timesteps
Episode finished after 20 timesteps
Episode finished after 14 timesteps
Episode finished after 12 timesteps
Episode finished after 95 timesteps
Episode finished after 38 timesteps


### Ejercicio 3

Se pide realizar un agente que aprenda a resolver alguno de los problemas toy_text incluidos en Gym (con representación discreta) usando Q-Learning: https://www.gymlibrary.dev/environments/toy_text/

Comenta el resultado obtenido y realiza pruebas (como en el ejercicio 2) para comprobar el comportamiento con distintos valores de los hiperparámetros. 

Es sencillo el entorno FrozenLake-v0  https://gym.openai.com/envs/FrozenLake-v0/ 

En este entorno el agente controla el movimiento de un personaje en un mundo de rejilla. Algunas baldosas son transitables (walkable) y otras hacen que el agente caiga al agua. La dirección de movimiento del agente es incierta y solo depende parcialmente de la dirección elegida (porque puede resbalar en el hielo). La recompensa se obtiene cuando el agente llega a traves de un camino transitable a una casilla objetivo. 

In [ ]:
env = gym.make('FrozenLake-v1', render_mode="human")
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        #env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info,X  = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Vamos a repetir el ejercicio 1 del Taxi, pero esta vez con el **entorno FrozenLake-v0**.

La mayoría del código es muy parecido. Pero hay que tener en cuenta que la recompensa es diferente. En el caso del Taxi, la recompensa es **-1** por cada movimiento, y **-10** por cada pickup y dropoff. Solo hay recompensa positiva cuando entrega el pasajero en el destino correcto. En el caso de FrozenLake, la recompensa por defecto es **1** si llega al objetivo, y **0** en cualquier otro caso.

Para mejorar el aprendizaje, ya que esta recompensa es muy mala, vamos a modificar el código para que la recompensa sea más parecido al problema de Taxi:

- +20 si llega al objetivo
- -10 si cae en el agua
- -1 por el resto de movimientos

Y **para que el problema no sea determinista**, configuramos para que `is_slippery = True` (posiblidad de deslizarse)

In [ ]:
import numpy as np

def evaluar_problema_lake(q_table, map_name="4x4", is_slippery = True):
    total_success, total_penalties, total_steps = 0, 0, 0
    episodes = 1000
    env = gym.make("FrozenLake-v1", desc=None, map_name=map_name, is_slippery=is_slippery).env

    for _ in range(episodes):
        env.reset()
        state = env.s

        done = False

        while not done:
            action = np.argmax(q_table[state])
            state, reward, done, info, X = env.step(action)
            
            if done and reward == 1:
                # Ha llegado al objetivo
                total_success += 1
            elif done and reward == 0:
                # Ha caído en un agujero
                total_penalties += 1

            total_steps += 1

    print(f"  Results after {episodes} episodes:")
    print(f"  Average steps per episode: {total_steps / episodes}")
    print(f"  Average success episode: {total_success / episodes}")
    print(f"  Average penalties per episode: {total_penalties / episodes}")

def probar_problema_lake(alpha, gamma, epsilon, rounds=10_000, map_name="4x4", is_slippery=True):
    import random
    from IPython.display import clear_output
    
    # Crear problema
    env = gym.make("FrozenLake-v1", desc=None, map_name=map_name, is_slippery=is_slippery)
    env.reset()

    q_table = np.zeros([env.observation_space.n, env.action_space.n])

    # For plotting metrics
    all_epochs = []

    initial_state = env.s
    for i in range(1, rounds):
        env.reset()
        state = env.s

        epochs, reward, penalties = 0, 0, 0
        done = False

        while not done:
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values

            next_state, reward, done, info, X = env.step(action) 

            # Modificamos el valor de reward para que sea más fácil aprender
            if done and reward == 1:
                # Ha llegado al objetivo
                reward = 20
            elif done and reward == 0:
                # Ha caído en un agujero
                reward = -10
                penalties += 1
            else:
                # Movimiento normal
                reward = -1

            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            state = next_state
            epochs += 1

    print(f"Training finished for initial state {initial_state} with alpha = {alpha}, gamma = {gamma}, epsilon = {epsilon}.")\
    
    # Imprimir el contenido de una celda aleatoria de la tabla Q
    print(f"  Q-table for initial state {initial_state} with alpha = {alpha}, gamma = {gamma}, epsilon = {epsilon}:")
    print(f"  {q_table[initial_state]}")

    evaluar_problema_lake(q_table, map_name, is_slippery)
    print()

Vamos a repetir el procedimiento realizado en el ejercicio 1, viendo cómo afecta al resultado tomando los valores extremos en los hiperparámetros.

Por defecto, el parámetro `is_slippery = True` (posiblidad de deslizarse), entonces no se va a alcanzar el 100% de éxito en el aprendizaje. Si ponemos `is_slippery = False` (sin posiblidad de deslizarse), entonces sí se alcanza el 100% de éxito en el aprendizaje.

### 1. Valores extremos de alpha

In [ ]:
probar_problema_lake(alpha=0.1, gamma=0.5, epsilon=0.5)

Training finished for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5:
  [-1.98625555 -1.99145294 -1.98547272 -1.99573358]
  Results after 1000 episodes:
  Average success episode: 0.667
  Average penalties per episode: 0.333



In [ ]:
probar_problema_lake(alpha=0.9, gamma=0.5, epsilon=0.5)

Training finished for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5:
  [-1.99999523 -1.99999949 -1.99999974 -1.99999782]
  Results after 1000 episodes:
  Average success episode: 0.811
  Average penalties per episode: 0.189



Al igual que el problema de Taxi, en algunas ejecuciones con `alpha = 0.1` se ha encontrado con el problema de bucle infinito porque la tabla Q no es muy fiable al aprender muy poco (valor de `alpha` pequeño)

En una de las ejecuciones, se ha obtenido el siguiente resultado:

```
Training finished for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5:
  [-1.98625555 -1.99145294 -1.98547272 -1.99573358]
  Results after 1000 episodes:
  Average success episode: 0.667
  Average penalties per episode: 0.333

Training finished for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5:
  [-1.99999523 -1.99999949 -1.99999974 -1.99999782]
  Results after 1000 episodes:
  Average success episode: 0.811
  Average penalties per episode: 0.189
```

Se puede ver que, en el caso de `alpha = 0.1` (explora menos), el agente tiene una tasa de éxito mucho menor que en el caso de `alpha = 0.9` (explora más, y conoce mejor la situación).

### 2. Valores extremos de gamma

In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.1, epsilon=0.5)

Training finished for initial state 0 with alpha = 0.5, gamma = 0.1, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.1, epsilon = 0.5:
  [-1.11111028 -1.111111   -1.11111058 -1.11111104]
  Results after 1000 episodes:
  Average success episode: 0.688
  Average penalties per episode: 0.312



In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.9, epsilon=0.5)

Training finished for initial state 0 with alpha = 0.5, gamma = 0.9, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.9, epsilon = 0.5:
  [-5.76443102 -7.57233639 -7.57850981 -7.49494571]
  Results after 1000 episodes:
  Average success episode: 0.657
  Average penalties per episode: 0.343



Al tomar valores extremos de gamma, el agente se fija más o menos en la recompensa futura. Podemos observar en este caso que ha pasado algo parecido que en el ejercicio de Taxi: al tomar valor muy pequeño de gamma, los valores resultantes de la tabla Q son muy parecidos (cercas del -1). 

```
Training finished for initial state 0 with alpha = 0.5, gamma = 0.1, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.1, epsilon = 0.5:
  [-1.11110954 -1.1111111  -1.11111111 -1.11111111]
  Results after 1000 episodes:
  Average timesteps per episode: 58.421
  Average penalties per episode: 0.31
```

La razón es la misma: la fórmula para actualizar Q table es `Q(s, a) = (1 - alpha) * Q(s, a) + alpha * (r + gamma * max(Q(s', a')))`, vemos que si el valor de `gamma` es muy pequeño, tenemos muy poco en cuenta la recompensa futura, y solo tenemos en cuenta la recompensa actual (`r`). 

Como la recompensa actual de movimiento es por defecto **-1**, al operar salen esos valores tan parecidos.

En cuanto a tasa de éxito, tras varias ejecuciones, no se ha podido extraer una conclusión que las diferencian. (No se ha encontrado un patrón claro)

### 3. Valores extremos de epsilon

In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.5, epsilon=0.1, rounds=100_000)

Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1:
  [-1.99253498 -1.9999247  -1.99577444 -1.99922364]
  Results after 1000 episodes:
  Average success episode: 0.532
  Average penalties per episode: 0.468



In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.5, epsilon=0.9, rounds=100_000)

Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9:
  [-1.99817464 -1.99636003 -1.99766964 -1.99893211]
  Results after 1000 episodes:
  Average success episode: 0.411
  Average penalties per episode: 0.589



Aquí hemos cambiado el número rondas de entrenamiento a 100 000 (10 veces más que antes), para que se note más la diferencia, ya que al ser un tablero de 4x4, el número de estados es mucho menor que el de Taxi.

En una de las ejecuciones hemos obtenido el siguiente resultado:

```
Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1:
  [-1.96956693 -1.99287223 -1.99790907 -1.9981825 ]
  Results after 1000 episodes:
  Average timesteps per episode: 50.364
  Average penalties per episode: 0.273
Tiempo: 1m 45s

Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9:
  [-1.98706892 -1.98271925 -1.98715014 -1.98904833]
  Results after 1000 episodes:
  Average timesteps per episode: 9.887
  Average penalties per episode: 0.837
Tiempo 25s
```

Es justo lo **contrario que en el problema de Taxi**: al tomar un valor muy pequeño de epsilon, el agente tarda más en terminar el entrenamiento, y al tomar un valor muy grande, el agente tarda menos en terminar el entrenamiento.

Pensamos que esto puede deberse a que, como el problema de Taxi es determinista (para cada estado, una acción determina siempre el mismo estado siguiente), lo que se aprendió puede volver a aplicarse en el mismo estado, y por lo tanto, usar datos de la tabla acelera el proceso.

Mientras que en el problema de FrozenLake, como tiene un factor de **aleatoriedad** (`is_slipery`), al realizar la misma acción, puede no llegar al mismo estado siguiente. Entonces lo que se guardó en Q table no ayuda tanto como en el problema de Taxi, y por este motivo, tarda más el entrenamiento que al escoger acciones aleatorias.

Para confirmar esta teoría, vamos a realizar el entrenamiento, pero esta vez con `is_slipery = False`, y veremos que ocurre:

In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.5, epsilon=0.1, rounds=100_000, is_slippery=False)

Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1:
  [-1.65625 -1.3125  -1.3125  -1.65625]
  Results after 1000 episodes:
  Average timesteps per episode: 6.0
  Average penalties per episode: 0.0



In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.5, epsilon=0.9, rounds=100_000, is_slippery=False)

Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9:
  [-1.65625 -1.3125  -1.3125  -1.65625]
  Results after 1000 episodes:
  Average timesteps per episode: 6.0
  Average penalties per episode: 0.0



En una de las ejecuciones se ha obtenido siguiente métrica:

- `epsilon = 0.1`: 17.3 segundos
- `epsilon = 0.9`: 23.2 segundos

Haciendo varias ejecuciones, el tiempo de entrenamiento con `epsilon = 0.9` es siempre superior al de `epsilon = 0.1`.

Por lo tanto, se confirma que **el factor de aleatoriedad hace que el entrenamiento sea más lento cuando epsilon es pequeño**.

En cuanto a la tasa de éxito, no se ha observado una diferencia enorme al variar el valor del epsilon.

----

Simplemente por curiosidad, vamos a ver si se mantiene las mismas conclusiones con el mapa de 8x8, y `is_slipery = True`.

In [ ]:
probar_problema_lake(alpha=0.1, gamma=0.5, epsilon=0.5, map_name="8x8", is_slippery=True, rounds=1000)

Training finished for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5:
  [-1.99999636 -1.99999797 -1.99999811 -1.99999881]
  Results after 1000 episodes:
  Average steps per episode: 111.308
  Average success episode: 0.009
  Average penalties per episode: 0.991



In [ ]:
probar_problema_lake(alpha=0.9, gamma=0.5, epsilon=0.5, map_name="8x8", is_slippery=True, rounds=1000)

Training finished for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5:
  [-1.99999981 -1.99999736 -1.99999967 -1.99999968]
  Results after 1000 episodes:
  Average steps per episode: 63.509
  Average success episode: 0.199
  Average penalties per episode: 0.801



Aquí la tasa de éxito es menor cuando alpha es grande, justo lo contrario que en el mapa de 4x4.

En una de las ejecuciones, se ha obtenido el siguiente resultado:

```
Training finished for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.1, gamma = 0.5, epsilon = 0.5:
  [-1.99999689 -1.99999507 -1.99999611 -1.99999859]
  Results after 1000 episodes:
  Average steps per episode: 98.527
  Average success episode: 0.222
  Average penalties per episode: 0.778

Training finished for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.9, gamma = 0.5, epsilon = 0.5:
  [-1.99998212 -1.99998821 -1.9999622  -1.99999646]
  Results after 1000 episodes:
  Average steps per episode: 77.911
  Average success episode: 0.254
  Average penalties per episode: 0.746
```

Se puede ver que la tasa de éxito es bastante pequeño en comparación con el mapa de 4x4, eso se debe a que el mapa es más grande, y el número de agujeros es mayor, junto con el factor de aleatoriedad, provoca que el agente cae más veces en agujeros.

En cuanto a los tiempos de entrenamiento, son muy parecidos en ambos casos, y no se ha encontrado una conclusión que las diferencie.

Y la tasa de éxito en ambos casos son parecidos.

In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.1, epsilon=0.5, map_name="8x8", is_slippery=True)

In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.9, epsilon=0.5, map_name="8x8", is_slippery=True)

Training finished for initial state 0 with alpha = 0.5, gamma = 0.9, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.9, epsilon = 0.5:
  [-9.65110098 -9.53332274 -9.63698221 -9.59529063]
  Results after 1000 episodes:
  Average success episode: 0.064
  Average penalties per episode: 0.936



Aquí pasa también que en caso de `gamma = 0.1`, los valores de la tabla Q son muy parecidos.

En una de las ejecuciones se ha obtenido el siguiente resultado:

```
Training finished for initial state 0 with alpha = 0.5, gamma = 0.1, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.1, epsilon = 0.5:
  [-1.11111111 -1.11111111 -1.11111111 -1.11111111]
  Results after 1000 episodes:
  Average success episode: 0.601
  Average penalties per episode: 0.399

Training finished for initial state 0 with alpha = 0.5, gamma = 0.9, epsilon = 0.5.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.9, epsilon = 0.5:
  [-9.65110098 -9.53332274 -9.63698221 -9.59529063]
  Results after 1000 episodes:
  Average success episode: 0.064
  Average penalties per episode: 0.936
```

Aquí se ha podido observar que la tasa del éxito cuando `gamma` es grande es menor que cuando `gamma` es pequeño. Concluímos que esto se debe a que, al tener un `gamma` grande, el agente se centra más en el futuro, y no en el presente. Al existe un factor de aleatoriedad, aunque el movimiento puede llevar al éxito, como se centra en el futuro éxito, no tiene tan en cuenta la posiblidad de resbalarse, y por eso obtenemos una tasa de éxito menor.

In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.5, epsilon=0.1, map_name="8x8")

Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.1:
  [-2. -2. -2. -2.]
  Results after 1000 episodes:
  Average success episode: 0.64
  Average penalties per episode: 0.36



In [ ]:
probar_problema_lake(alpha=0.5, gamma=0.5, epsilon=0.9, map_name="8x8")

Training finished for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9.
  Q-table for initial state 0 with alpha = 0.5, gamma = 0.5, epsilon = 0.9:
  [-1.99999582 -1.99999445 -1.99999354 -1.99999313]
  Results after 1000 episodes:
  Average success episode: 0.216
  Average penalties per episode: 0.784



De nuevo se puede observar que tarda más el entrenamiento cuando epsilon es pequeño cuando existe un factor de aleatoriedad.

--------------------

#### Ejercicio 4.  Opcional.   
Aplica el algoritmo Q-learning para diseñar un agente que aprenda a resolver alguno de los puzles que tienes resueltos en la práctica 1 (puzle de 8, jarras, misioneros,..). 
Comenta el resultado comparandolo con el comportamiento del agente que ya tienes hecho de la práctica anterior que resuelve el problema usando búsqueda en espacio de estados.  
Discute claramente las ventajas e inconvenientes con las métricas y resultados obtenidos.

Si no resuelves el problema puedes hacer el razonamiento teórico de cómo se comportaría el agente con Q-learning si lo comparamos con un agente que resuelve el problema con búsqueda en el espacio de estados. Indica las ventajas e inconvenientes de las dos aproximaciones. 

Nosotros hemos decidido resolver el problema de los misioneros. Recordamos en que consiste el problema

> Hay 3 misioneros y 3 caníbales en la orilla de un rio. Tanto los misioneros como los caníbales quieren cruzar a la otra orilla utilizando una barca. En la barca solo pueden viajar dos personas a la vez. En una orilla no puede haber misioneros si los canibales los superan en número, pues sino los caníbales se comerán a los misioneros. Encuentra una manera de hacer cruzar a todos los misioneros y caníbales a la otra orilla.

In [ ]:
# Realizamos un par de imports que vamos a utilizar a partir de ahora
from dataclasses import dataclass
from copy import deepcopy
import numpy as np
import random

Para hablar de algo, primero tenemos que ponerle nombre. Por lo tanto, antes de intentar resolver el problema, tenemos que ser capaces de representarlo en código.

Definimos el estado del problema como una tupla con 2 enteros y un booleano. Los enteros indican el número de misioneros y caníbales que hay en la orilla original, y el booleano indica si la barca se encuentra en la orilla opuesta.

In [ ]:
@dataclass
class Estado:
    misioneros: int
    canibales:  int
    barca_en_la_otra_orilla: bool

estado_inicial = Estado(3,3,False)
estado_final   = Estado(0,0,True)

Las acciones que podemos realizar en este problema se representan con una tupla de dos enteros, que indican el número de misioneros y caníbales, respectivamente, que se suben a la barca. No es necesario describir la dirección
de la barca, ya que ésta la podemos obtener del estado. Al fin y al cabo, solo hay dos orillas entre las que la barca se puede mover, así que si la barca está en una orilla, solo se puede mover a la otra

In [ ]:
@dataclass
class Accion:
    misioneros: int
    canibales:  int
acciones = [
    Accion(misioneros=1,canibales=0),
    Accion(misioneros=0,canibales=1),
    Accion(misioneros=1,canibales=1),
    Accion(misioneros=2,canibales=0),
    Accion(misioneros=0,canibales=2)
]

Una vez tenemos el estado y las acciones del estado, procedemos a codificarlos con un número. De esta manera, nos resultará más fácil trabajar con ellos cuando hagamos el Q-Learning.

In [ ]:
def encode_state(s: Estado) -> int:
    return s.misioneros + 4*s.canibales + 4*4*s.barca_en_la_otra_orilla
def decode_state(s: int) ->  Estado:
    return Estado(s%4, (s//4)%4, (s//4**2) == 1)

def encode_action(a: Accion) -> int:
    return acciones.index(a)
def decode_action(a: int) -> Accion:
    return acciones[a]

Codificar las acciones ha sido más fácil, pues tenemos una lista finita de ellas. Podemos usar simplemente el índice que tienen en la lista de acciones como su codificación, y así tenemos un intervalo de enteros en los que todo número se corresponde con una acción. Sin embargo, no ocurre lo mismo en el caso de los estados. Hay estados que no pueden darse dado un estado inicial, como por ejemplo `(3,3,True)`. En este caso no importa demasiado, pues el problema es pequeño, y si a un estado no podemos acceder, simplemente podemos ignorar sus valores en la Q-table. Sin embargo, para problemas más grandes, esto puede ser una causa de ineficiencia, así que hay que tenerlo en cuenta.

In [ ]:
print(f"{encode_state(Estado(3,3,1))+1}⨯{len(acciones)}")

32⨯5


Así, podemos representar nuestra Q-table como una matrix de dimensiones $31\times5$

In [ ]:
q_table = [[0]*5]*32

Una vez hemos definido el estado y las acciones, falta definir que sucede cuando realizamos una acción sobre un estado concreto

In [ ]:
def realizar(estado_codificado: int, accion_codificada: int) -> int:
    s = decode_state(estado_codificado)
    a = decode_action(accion_codificada)
    if s.barca_en_la_otra_orilla:
        return encode_state(Estado(
            misioneros = s.misioneros + a.misioneros,
            canibales  = s.canibales  + a.canibales,
            barca_en_la_otra_orilla = False
        ))
    else:
        return encode_state(Estado(
            misioneros = s.misioneros - a.misioneros,
            canibales  = s.canibales  - a.canibales,
            barca_en_la_otra_orilla = True
        ))

Y ya estamos casi ahí. Ahora solo nos falta la parte más importante, la función de recompensa. 

Esta es la parte más libre de la implementación. Debemos elegir que acciones recompensar, en distintas situaciones, y por cuanto recompensarlas.
 - Primero, queremos penalizar acciones ilegales. Por ejemplo, las que provocan que los caníbales se coman a los misioneros, o peor, las que meten a más misioneros o caníbales en la barca de los que hay en la orilla en la que está la barca. En la función `realizar` decidimos no reflejar que estas acciones son ilegales, pero indicamos su ilegalidad en la función de recompensa.
   Esta penalización será el valor `illegal_penalty`.
 - Cuando el agente lleve a una persona a la otra orilla, le premiaremos.  
   Esta recompensa será el valor `person_crossed_reward`.
 - Penalizaremos al agente cuando se lleve una persona de la otra orilla de vuelta al inicio.  
   Esta penalización será el valor `person_returned_penalty`.


In [ ]:
illegal_penalty         = -100
person_returned_penalty =  -12
person_crossed_reward   =   10

def accion_ilegal(s: Estado, a: Accion, explain = False) ->  bool:

    if s.barca_en_la_otra_orilla:
        misioneros_aqui = s.misioneros + a.misioneros
        canibales_aqui  = s.canibales  + a.canibales
        canibales_orilla_destino  = 3 - s.canibales
        misioneros_orilla_destino = 3 - s.misioneros
    else:
        misioneros_aqui = s.misioneros - a.misioneros
        
        canibales_aqui  = s.canibales  - a.canibales

        canibales_orilla_destino  = s.canibales
        misioneros_orilla_destino = s.misioneros
    misioneros_alla = 3 - misioneros_aqui
    canibales_alla  = 3 - canibales_aqui


    if misioneros_aqui < canibales_aqui and misioneros_aqui != 0:
        if explain: print("En esta orilla hay más caníbales que misioneros")
        return True
    if misioneros_alla < canibales_alla and misioneros_alla != 0:
        if explain: print("En la otra orilla hay más caníbales que misioneros")
        return True
    if canibales_orilla_destino < a.canibales:
        if explain: print("Trajimos más canibales de la otra orilla de los que había")
        return True
    if misioneros_orilla_destino < a.misioneros:
        if explain: print("Trajimos más misioneros de la otra orilla de los que había")
        return True

    return False

def recompensa(estado_codificado: int, accion_codificada: int, explain = False) -> int:
    s = decode_state(estado_codificado)
    a = decode_action(accion_codificada)
    if accion_ilegal(s,a, explain):
        return illegal_penalty
    personas_en_barca = a.misioneros + a.canibales
    if s.barca_en_la_otra_orilla:
        if explain: print(f"Penalización por traer {personas_en_barca} persona{'' if personas_en_barca == 1 else 's'} en barca")
        return person_returned_penalty*personas_en_barca
    else:
        if explain: print(f"Recompensa por llevar {personas_en_barca} persona{'' if personas_en_barca == 1 else 's'} en barca")
        return person_crossed_reward*personas_en_barca

Ya tenemos todos los componentes necesarios para resolver el problema con Q-Learning. Solo falta implementar el algoritmo.

In [ ]:
def visualizar_estado(estado_codificado: int):
    s = decode_state(estado_codificado)
    lhs = ('m'*s.misioneros).ljust(3) + " " + ('c'*s.canibales).ljust(3)
    rhs = ('m'*(3-s.misioneros)).ljust(3) +  " " + ('c'*(3-s.canibales)).ljust(3)
    return lhs + ' │ ' + rhs
mostrar_estado = lambda e : print(visualizar_estado(e))

def visualizar_accion(accion_codificada: int):
    accion = decode_action(accion_codificada)
    return 'm'*accion.misioneros + 'c'*accion.canibales
mostrar_accion = lambda a : print(visualizar_accion(a))

In [ ]:
def q_learning(initial: Estado, alpha = 0.8, gamma = 0.8, epsilon = 0.8, epochs = 10_000, verbose = False, metrics = True):
    q_table = np.zeros((32,5))
    meta = encode_state(estado_final)
    info_de_entrenamiento = []
    
    for epoch in range(epochs):
        estado = encode_state(initial)
        puntuacion_final = 0
        turnos = 0
        
        if metrics or verbose: print(f"Epoch {epoch}",end=": \n" if verbose else " ")
        if verbose: mostrar_estado(estado)
        while estado != meta:
            if random.uniform(0, 1) < epsilon:
                if verbose: print("Decidimos explorar un poco")
                accion = encode_action(random.choice(acciones)) # Explora el espacio de acciones
            else:
                if verbose: print("Decidimos usar el conocimiento adquirido")
                accion = np.argmax(q_table[estado]) # Explota el conocimiento pasado
            if verbose: print(f"Movemos ",end=""); mostrar_accion(accion)
                
            puntuacion = recompensa(estado, accion, explain=verbose)
            estado_siguiente = realizar(estado, accion)
            if verbose: mostrar_estado(estado_siguiente)

            q_value_antiguo = q_table[estado, accion]
            q_value_maximo  = np.max(q_table[estado_siguiente])
            q_value_nuevo   = (1 - alpha) * q_value_antiguo + alpha * (puntuacion + gamma * q_value_maximo)
            q_table[estado, accion] = q_value_nuevo

            estado = estado_siguiente
            puntuacion_final += puntuacion
            turnos += 1
        if metrics or verbose: print(f"realiza {turnos} turnos, con una putuación final de {puntuacion_final}")
        info_de_entrenamiento.append((turnos,puntuacion_final))        
    
    return q_table, info_de_entrenamiento

Ahora, podemos pasar a resolver el problema. Para ello, solo tenemos que decidir el valor de los hiperparámetros $\alpha$, $\gamma$ y $\epsilon$. 

In [ ]:
q_table, info = q_learning(estado_inicial, alpha=0.8, gamma=0.8, epsilon=0.8, verbose = False)

Epoch 0 realiza 33 turnos, con una putuación final de -2084
Epoch 1 realiza 40 turnos, con una putuación final de -1306
Epoch 2 realiza 2 turnos, con una putuación final de -200
Epoch 3 realiza 26 turnos, con una putuación final de -816
Epoch 4 realiza 36 turnos, con una putuación final de -1936
Epoch 5 realiza 4 turnos, con una putuación final de -280
Epoch 6 realiza 139 turnos, con una putuación final de -7010
Epoch 7 realiza 11 turnos, con una putuación final de -290
Epoch 8 realiza 25 turnos, con una putuación final de -1482
Epoch 9 realiza 13 turnos, con una putuación final de -796
Epoch 10 realiza 42 turnos, con una putuación final de -1976
Epoch 11 realiza 25 turnos, con una putuación final de -1192
Epoch 12 realiza 19 turnos, con una putuación final de -958
Epoch 13 realiza 37 turnos, con una putuación final de -2058
Epoch 14 realiza 17 turnos, con una putuación final de -1218
Epoch 15 realiza 8 turnos, con una putuación final de -352
Epoch 16 realiza 5 turnos, con una putuació

Epoch 488 realiza 20 turnos, con una putuación final de -1124
Epoch 489 realiza 12 turnos, con una putuación final de -380
Epoch 490 realiza 8 turnos, con una putuación final de -264
Epoch 491 realiza 10 turnos, con una putuación final de -682
Epoch 492 realiza 10 turnos, con una putuación final de -376
Epoch 493 realiza 38 turnos, con una putuación final de -1820
Epoch 494 realiza 40 turnos, con una putuación final de -2248
Epoch 495 realiza 50 turnos, con una putuación final de -2206
Epoch 496 realiza 58 turnos, con una putuación final de -3796
Epoch 497 realiza 11 turnos, con una putuación final de -368
Epoch 498 realiza 7 turnos, con una putuación final de -164
Epoch 499 realiza 30 turnos, con una putuación final de -1018
Epoch 500 realiza 59 turnos, con una putuación final de -3108
Epoch 501 realiza 6 turnos, con una putuación final de -392
Epoch 502 realiza 8 turnos, con una putuación final de -472
Epoch 503 realiza 13 turnos, con una putuación final de -534
Epoch 504 realiza 21 

Epoch 955 realiza 48 turnos, con una putuación final de -2438
Epoch 956 realiza 57 turnos, con una putuación final de -2442
Epoch 957 realiza 3 turnos, con una putuación final de -60
Epoch 958 realiza 5 turnos, con una putuación final de -62
Epoch 959 realiza 30 turnos, con una putuación final de -1480
Epoch 960 realiza 7 turnos, con una putuación final de -262
Epoch 961 realiza 29 turnos, con una putuación final de -1544
Epoch 962 realiza 6 turnos, con una putuación final de -294
Epoch 963 realiza 18 turnos, con una putuación final de -1090
Epoch 964 realiza 41 turnos, con una putuación final de -1364
Epoch 965 realiza 37 turnos, con una putuación final de -1564
Epoch 966 realiza 9 turnos, con una putuación final de -482
Epoch 967 realiza 26 turnos, con una putuación final de -1278
Epoch 968 realiza 29 turnos, con una putuación final de -1432
Epoch 969 realiza 33 turnos, con una putuación final de -856
Epoch 970 realiza 30 turnos, con una putuación final de -1294
Epoch 971 realiza 13 

Epoch 1409 realiza 5 turnos, con una putuación final de -260
Epoch 1410 realiza 18 turnos, con una putuación final de -958
Epoch 1411 realiza 28 turnos, con una putuación final de -1134
Epoch 1412 realiza 22 turnos, con una putuación final de -874
Epoch 1413 realiza 35 turnos, con una putuación final de -1530
Epoch 1414 realiza 27 turnos, con una putuación final de -1696
Epoch 1415 realiza 10 turnos, con una putuación final de -596
Epoch 1416 realiza 17 turnos, con una putuación final de -464
Epoch 1417 realiza 25 turnos, con una putuación final de -1112
Epoch 1418 realiza 6 turnos, con una putuación final de -404
Epoch 1419 realiza 12 turnos, con una putuación final de -566
Epoch 1420 realiza 5 turnos, con una putuación final de -52
Epoch 1421 realiza 8 turnos, con una putuación final de -604
Epoch 1422 realiza 35 turnos, con una putuación final de -1532
Epoch 1423 realiza 19 turnos, con una putuación final de -460
Epoch 1424 realiza 9 turnos, con una putuación final de -452
Epoch 142

Epoch 1879 realiza 23 turnos, con una putuación final de -1398
Epoch 1880 realiza 5 turnos, con una putuación final de -52
Epoch 1881 realiza 6 turnos, con una putuación final de -304
Epoch 1882 realiza 13 turnos, con una putuación final de -686
Epoch 1883 realiza 18 turnos, con una putuación final de -780
Epoch 1884 realiza 45 turnos, con una putuación final de -2342
Epoch 1885 realiza 16 turnos, con una putuación final de -526
Epoch 1886 realiza 4 turnos, con una putuación final de -170
Epoch 1887 realiza 24 turnos, con una putuación final de -1108
Epoch 1888 realiza 19 turnos, con una putuación final de -728
Epoch 1889 realiza 9 turnos, con una putuación final de -582
Epoch 1890 realiza 4 turnos, con una putuación final de -192
Epoch 1891 realiza 40 turnos, con una putuación final de -2138
Epoch 1892 realiza 10 turnos, con una putuación final de -366
Epoch 1893 realiza 31 turnos, con una putuación final de -1446
Epoch 1894 realiza 49 turnos, con una putuación final de -1992
Epoch 18

...

Epoch 7905 realiza 17 turnos, con una putuación final de -682
Epoch 7906 realiza 34 turnos, con una putuación final de -1486
Epoch 7907 realiza 9 turnos, con una putuación final de -582
Epoch 7908 realiza 38 turnos, con una putuación final de -1750
Epoch 7909 realiza 38 turnos, con una putuación final de -1724
Epoch 7910 realiza 30 turnos, con una putuación final de -1754
Epoch 7911 realiza 3 turnos, con una putuación final de -60
Epoch 7912 realiza 22 turnos, con una putuación final de -538
Epoch 7913 realiza 48 turnos, con una putuación final de -1854
Epoch 7914 realiza 26 turnos, con una putuación final de -1156
Epoch 7915 realiza 49 turnos, con una putuación final de -2088
Epoch 7916 realiza 10 turnos, con una putuación final de -148
Epoch 7917 realiza 6 turnos, con una putuación final de -282
Epoch 7918 realiza 7 turnos, con una putuación final de -350
Epoch 7919 realiza 24 turnos, con una putuación final de -1296
Epoch 7920 realiza 6 turnos, con una putuación final de -316
Epoch 

Epoch 8430 realiza 19 turnos, con una putuación final de -864
Epoch 8431 realiza 6 turnos, con una putuación final de -184
Epoch 8432 realiza 35 turnos, con una putuación final de -1390
Epoch 8433 realiza 2 turnos, con una putuación final de -90
Epoch 8434 realiza 27 turnos, con una putuación final de -1378
Epoch 8435 realiza 36 turnos, con una putuación final de -1892
Epoch 8436 realiza 41 turnos, con una putuación final de -1826
Epoch 8437 realiza 25 turnos, con una putuación final de -560
Epoch 8438 realiza 22 turnos, con una putuación final de -854
Epoch 8439 realiza 17 turnos, con una putuación final de -932
Epoch 8440 realiza 5 turnos, con una putuación final de -62
Epoch 8441 realiza 17 turnos, con una putuación final de -968
Epoch 8442 realiza 40 turnos, con una putuación final de -2040
Epoch 8443 realiza 46 turnos, con una putuación final de -2412
Epoch 8444 realiza 53 turnos, con una putuación final de -2814
Epoch 8445 realiza 14 turnos, con una putuación final de -490
Epoch 

Epoch 8846 realiza 6 turnos, con una putuación final de -284
Epoch 8847 realiza 9 turnos, con una putuación final de -212
Epoch 8848 realiza 24 turnos, con una putuación final de -932
Epoch 8849 realiza 8 turnos, con una putuación final de -68
Epoch 8850 realiza 6 turnos, con una putuación final de -294
Epoch 8851 realiza 12 turnos, con una putuación final de -872
Epoch 8852 realiza 15 turnos, con una putuación final de -460
Epoch 8853 realiza 10 turnos, con una putuación final de -792
Epoch 8854 realiza 78 turnos, con una putuación final de -2494
Epoch 8855 realiza 4 turnos, con una putuación final de -280
Epoch 8856 realiza 29 turnos, con una putuación final de -1148
Epoch 8857 realiza 4 turnos, con una putuación final de -170
Epoch 8858 realiza 34 turnos, con una putuación final de -1180
Epoch 8859 realiza 5 turnos, con una putuación final de -172
Epoch 8860 realiza 13 turnos, con una putuación final de -546
Epoch 8861 realiza 19 turnos, con una putuación final de -926
Epoch 8862 re

Epoch 9244 realiza 9 turnos, con una putuación final de -300
Epoch 9245 realiza 7 turnos, con una putuación final de -296
Epoch 9246 realiza 34 turnos, con una putuación final de -1782
Epoch 9247 realiza 7 turnos, con una putuación final de -372
Epoch 9248 realiza 47 turnos, con una putuación final de -2454
Epoch 9249 realiza 90 turnos, con una putuación final de -4144
Epoch 9250 realiza 49 turnos, con una putuación final de -2298
Epoch 9251 realiza 2 turnos, con una putuación final de -200
Epoch 9252 realiza 19 turnos, con una putuación final de -676
Epoch 9253 realiza 6 turnos, con una putuación final de -282
Epoch 9254 realiza 42 turnos, con una putuación final de -1792
Epoch 9255 realiza 48 turnos, con una putuación final de -1934
Epoch 9256 realiza 21 turnos, con una putuación final de -1248
Epoch 9257 realiza 9 turnos, con una putuación final de -680
Epoch 9258 realiza 64 turnos, con una putuación final de -3456
Epoch 9259 realiza 20 turnos, con una putuación final de -894
Epoch 

Epoch 9892 realiza 8 turnos, con una putuación final de -276
Epoch 9893 realiza 2 turnos, con una putuación final de -90
Epoch 9894 realiza 28 turnos, con una putuación final de -1346
Epoch 9895 realiza 3 turnos, con una putuación final de -60
Epoch 9896 realiza 12 turnos, con una putuación final de -272
Epoch 9897 realiza 7 turnos, con una putuación final de -174
Epoch 9898 realiza 4 turnos, con una putuación final de -280
Epoch 9899 realiza 11 turnos, con una putuación final de -552
Epoch 9900 realiza 29 turnos, con una putuación final de -1476
Epoch 9901 realiza 17 turnos, con una putuación final de -900
Epoch 9902 realiza 24 turnos, con una putuación final de -986
Epoch 9903 realiza 15 turnos, con una putuación final de -218
Epoch 9904 realiza 20 turnos, con una putuación final de -1200
Epoch 9905 realiza 6 turnos, con una putuación final de -174
Epoch 9906 realiza 8 turnos, con una putuación final de -252
Epoch 9907 realiza 4 turnos, con una putuación final de -180
Epoch 9908 real

Una vez hemos "aprendido" la Q-table, podemos usarla para resolver el problema

In [ ]:
def traversar(s: Estado, q_table: np.ndarray, verbose = False):
    puntuacion_final, turnos = 0, 0
    
    estado = encode_state(s)
    meta   = encode_state(estado_final)
    if verbose: mostrar_estado(estado)
    while estado != meta:
        accion = np.argmax(q_table[estado])
        if verbose: 
            if decode_state(estado).barca_en_la_otra_orilla:
                print("←".rjust(8)+visualizar_accion(accion))
            else:
                print(visualizar_accion(accion).rjust(8+1)+ "→")
                
    
        puntuacion = recompensa(estado, accion)
        estado = realizar(estado, accion)
        if verbose: mostrar_estado(estado)

        puntuacion_final += puntuacion
        turnos += 1
    if verbose: print(f"Resolvimos el problema con {turnos} turnos y una puntuación de {puntuacion_final}")
    return puntuacion_final, turnos

In [ ]:
puntuacion, turnos = traversar(estado_inicial, q_table, verbose = True)

mmm ccc │        
       mc→
mm  cc  │ m   c  
       ←m
mmm cc  │     c  
       cc→
mmm     │     ccc
       ←c
mmm c   │     cc 
       mm→
m   c   │ mm  cc 
       ←mc
mm  cc  │ m   c  
       mm→
    cc  │ mmm c  
       ←c
    ccc │ mmm    
       cc→
    c   │ mmm cc 
       ←m
m   c   │ mm  cc 
       mc→
        │ mmm ccc
Resolvimos el problema con 11 turnos y una puntuación de 48


Como observamos, ¡el algoritmo ha conseguido aprender una solución al problema! 